<h1 align='center'>BitCoin Data Transformation Internal</h1>

In [1]:
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
import csv

client = MongoClient("129.26.78.40:27017")
db = client['BitCoin']

## Aggregate the BitCoin amount per transaction per address

In [2]:
for i in range(1, 20):
    print "blocks aggregate " + str(i)
    db["blocks_{0}".format(i)].aggregate([
                                {"$unwind":"$tx"},
                                 {"$project":
                                         {"inputs":"$tx.inputs",
                                          "out":"$tx.out",
                                          "tx_time":"$tx.time",
                                          "tx_hash": "$tx.hash",
                                          "bl_time": "$time",
                                          "_id":0
                                         }},
                                 {"$unwind":"$inputs"},
                                 {"$project":
                                            {"bl_time": 1,
                                             "tx_time":"$tx_time",
                                             "tx_hash": "$tx_hash",
                                             "address_source":"$inputs.prev_out.addr",
                                             "amount":"$inputs.prev_out.value",
                                             "out":"$out"}},
                                 {"$unwind":"$out"},
                                 {"$group":
                                          {
                                            "_id": {"tx_hash": "$tx_hash", "sender":"$address_source", "bl_time": "$bl_time", "tx_time": "$tx_time", "reciever":"$out.addr"},
                                                "amount":{ "$sum": "$out.value"}

                                            }
                                 },
                                {"$match": { "amount": {"$gt": 0}}},
                                {"$out":"blocks_{0}_aggregation".format(i)}],allowDiskUse=True)


SyntaxError: invalid syntax (<ipython-input-2-0a1f21a34173>, line 1)

## Collect all aggrefation into one collection

In [ ]:
for i in range(1, 20):
    collection = "blocks_{0}_aggregation".format(i)
    data = db[collection].find({})
    db["blocks_aggregation"].insert_many(data)
db["blocks_aggregation"].create_index([("tx_time",1)])

## Create Separate Collection for 2017

In [ ]:
db["blocks_aggregation"].aggregate([ {"$match": { "tx_time": {"$gte": datetime.datetime(2017, 1, 1)}}}, 
                                     {"$project": {
                                            "_id": 1,
                                            "amount": 1,
                                            "tx_time": 1,
                                            "week": {"$week": "$tx_time"}
                                        }
                                     },
                                      {"$out":"blocks_aggregation_2017"}])
db["blocks_aggregation_2017"].create_index([("tx_time",1)])

## Split the senders weekly

In [ ]:
for week in range(1, 33):
    db["blocks_aggregation_2017"].aggregate([ {"$match": { "week": week}},
                        {"$group": { 
                                     "_id": "$_id.sender", 
                                      "tx": {"$addToSet": {"h":"$_id.tx_hash",
                                                           "r": "$_id.reciever",
                                                           "ts": "$_id.time",
                                                           "t": "$tx_time",
                                                           "a": "$amount"}
                                            },
                                       "ntx": {"$sum": 1}
                                                }
                                    },
                                 {"$out":"senders_2017_week_{0}".format(week)}
                                 ],
                            allowDiskUse=True)
    db["senders_2017_week_{0}".format(week)].create_index([("tx.ts",1)])

## Time Series for # of Transactions per Second

## Time Series for # of Edges per Second

In [225]:
a = db["blocks_aggregation_2017"].aggregate([#{"$limit": 10000},
                                             {"$group": {
                                                 "_id": {"tx_time": "$tx_time", "time": "$_id.time"}
                                                     }
                                             },
                                            {"$group": {
                                                        "_id": {
                                                                    "month": {"$month": "$_id.tx_time"}, 
                                                                    "week": {"$week": "$_id.tx_time"}, 
                                                                    "day": {"$dayOfYear": "$_id.tx_time"}, 
                                                                    "hour": {"$hour": "$_id.tx_time"},
                                                                    "minutes": {"$minute":"$_id.tx_time"},
#                                                                     "second": {"$second":"$_id.tx_time"},
                                                                  
                                                                },
                                                        "txs":{"$addToSet": "$_id.time"},
                                                        "number": {"$sum": 1}
                                                        }
                                             },
                                             {"$project": {
                                                     "_id": 0,
                                                     "date": "$_id.date",
                                                     "number": 1,
                                                     "month": "$_id.month",
                                                     "week": "$_id.week",
                                                     "day": "$_id.day",
                                                     "hour": "$_id.hour",
                                                     "minutes": "$_id.minutes",
                                                     "second": "$_id.second",
                                                     "txs": "$txs"
                                                 }
                                             },
                                            {"$out":"number_of_edges_per_second_2017"}
                                        ], allowDiskUse=True)

In [223]:
a.next()

{u'day': 3,
 u'hour': 16,
 u'minutes': 41,
 u'month': 1,
 u'number': 2,
 u'txs': [1483461698, 1483461697],
 u'week': 1}

## Data for Ernane

In [151]:
def query_edges_per_day(week, day_of_year):
    cursor = db["senders_2017_week_{0}".format(week)].aggregate([
                                            {"$unwind": "$tx"},
                                            {"$project": {
                                                  "_id": 0,
                                                    "s": "$_id",
                                                    "r": "$tx.r",
                                                    "a": "$tx.a",
                                                    "ts": "$tx.ts",
                                                    "day": {"$dayOfYear": "$tx.t"}
                                                }
                                            },
                                            {"$match": { "day" : day_of_year}},
                                            {"$sort": {"ts": 1}},

    ],   allowDiskUse=True)
    return cursor

In [155]:
def export_cursor_to_csv(cursor, file_name, fields):
    with open(file_name, 'w') as outfile:        
        writer = csv.DictWriter(outfile, fieldnames=fields, delimiter='\t')
        writer.writeheader()
        for x in cursor:
            writer.writerow(x)

In [156]:
fields = ['s', 'r', 'a', 'ts', 'day']
for week in range(1, 33):
    for day_of_week in range(1, 8):
        day_of_year = day_of_week + ((week - 1) * 7)
        cursor = query_edges_per_day(week, day_of_year)
        export_cursor_to_csv(cursor, 'ernane_2017_day_{0}.csv'.format(day_of_year), fields)

KeyboardInterrupt: 

In [16]:
db['senders_2017_week_11'].aggregate([#{"$match": {"_id": "1dice8EMZmqKvrGE4Qc9bUFf9PX3xaYDp"}},
                                            {"$match": {"ntx": {"$gte": 10}}},
                                            {"$unwind": "$tx"},
                                            {"$group": {
                                                "_id": {"s": "$_id", "ts": "$tx.ts" },
                                            }},
                                            {"$group": {
                                                "_id": "$_id.s",
                                                "txs": {"$addToSet": "$_id.ts"},
                                                "ntx": {"$sum": 1}
                                            }},
                                            {"$match": {"ntx": {"$gte": 10}}}
                                        ],
                                allowDiskUse=True).next()

StopIteration: 

In [ ]:
db["number_of_edges_per_second_2017"].create_index([("day",1)])

In [3]:
db[u'number_of_edges_per_second_2017'].find({}).limit(1).next()

{u'_id': ObjectId('59bff47348ec0fd6e85070f8'),
 u'day': 3,
 u'hour': 17,
 u'minutes': 0,
 u'month': 1,
 u'number': 3,
 u'txs': [1483462806, 1483462855, 1483462817],
 u'week': 1}